In [1]:
#Mouting drive for getting access to MyDrive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#Installing pyspark
!pip install pyspark

     |████████████████████████████████| 204.2MB 63kB/s 
     |████████████████████████████████| 204kB 40.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=dee1f3e24cb138e54a86e85c9500e4bdd515d1ff168723935e6bef37ff537a46
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
#Importing required libraries
import pyspark

In [4]:
from pyspark import SparkContext
#to stop the sparkcontext if its already running
#sc.stop()
#setting sparkcontext locally
sc = SparkContext("local", "rest_app")

In [5]:
#creating sqlcontext for accessing SparkSQL
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [6]:
#defining schema for the data as required within dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, StringType
schema = StructType([\
        StructField("user_id_key", IntegerType()),\
        StructField("business_id_key", IntegerType()),\
        StructField("stars", FloatType()),\
        StructField("business_name", StringType()),\
    ])

In [7]:
#creating sparksql dataframe with set schema from ratings.csv file
spark_df = sqlContext.read.csv("drive/My Drive/CMPE-256-Project/MonYelpMergedData(final_processed).csv", header=True, sep=",", schema=schema)

In [8]:
#displaying the head part to check whether dataframe is created properly or not
spark_df.head()

Row(user_id_key=0, business_id_key=0, stars=5.0, business_name="Dunn's Famous")

In [72]:
#displaying first 20 rows from dataframe
spark_df.head(60)

[Row(user_id_key=0, business_id_key=0, stars=5.0, business_name="Dunn's Famous"),
 Row(user_id_key=1, business_id_key=1, stars=5.0, business_name='Copper Branch'),
 Row(user_id_key=2, business_id_key=2, stars=3.0, business_name='Stash Café'),
 Row(user_id_key=3, business_id_key=0, stars=5.0, business_name="Dunn's Famous"),
 Row(user_id_key=4, business_id_key=3, stars=4.0, business_name='La Société Montréal'),
 Row(user_id_key=5, business_id_key=4, stars=4.0, business_name='Cacao 70'),
 Row(user_id_key=6, business_id_key=5, stars=3.0, business_name='La Chronique'),
 Row(user_id_key=6, business_id_key=6, stars=4.0, business_name='Toi Moi & Café'),
 Row(user_id_key=7, business_id_key=7, stars=5.0, business_name='Saint-Laurent Warehouse'),
 Row(user_id_key=8, business_id_key=8, stars=4.0, business_name='Cacao 70'),
 Row(user_id_key=8, business_id_key=9, stars=4.0, business_name='Cacao 70'),
 Row(user_id_key=8, business_id_key=10, stars=3.0, business_name='Pasta Casareccia'),
 Row(user_id_k

In [10]:
#checkig the datatype of each attribute
spark_df.dtypes

[('user_id_key', 'int'),
 ('business_id_key', 'int'),
 ('stars', 'float'),
 ('business_name', 'string')]

In [11]:
spark_df.count()

3437

In [12]:
##split the ratings into training and testing data
training, testing = spark_df.randomSplit([0.9, 0.1], seed=12345)

In [13]:
training.dtypes

[('user_id_key', 'int'),
 ('business_id_key', 'int'),
 ('stars', 'float'),
 ('business_name', 'string')]

In [14]:
training.count()

3129

In [15]:
testing.count()

308

In [16]:
training.head(3)

[Row(user_id_key=0, business_id_key=0, stars=5.0, business_name="Dunn's Famous"),
 Row(user_id_key=1, business_id_key=1, stars=5.0, business_name='Copper Branch'),
 Row(user_id_key=2, business_id_key=2, stars=3.0, business_name='Stash Café')]

In [ ]:
#saving training data to csv file in training.csv folder ,using repartition() to save all the data samples in single csv file
csv_location = 'drive/My Drive/CMPE-256-Project/mon_rest_training.csv'
training.repartition(1).write.csv(path=csv_location, mode="append", header="true")

In [ ]:
#saving testing data to csv file in testing.csv folder ,using repartition() to save all the data samples in single csv file
csv_location = 'drive/My Drive/CMPE-256-Project/mon_rest_testing.csv'
testing.repartition(1).write.csv(path=csv_location, mode="append", header="true")

In [17]:
#Importing the required libraries
import numpy as np 
import pandas as pd

In [ ]:
##Visualizing data for further analysis

In [18]:
training_df = pd.read_csv("drive/My Drive/CMPE-256-Project/mon_rest_training.csv")

In [19]:
training_df.head(20)

,user_id_key,business_id_key,stars,business_name
0,0,0,5.0,Dunn's Famous
1,1,1,5.0,Copper Branch
2,2,2,3.0,Stash Café
3,3,0,5.0,Dunn's Famous
4,4,3,4.0,La Société Montréal
5,5,4,4.0,Cacao 70
6,6,5,3.0,La Chronique
7,6,6,4.0,Toi Moi & Café
8,7,7,5.0,Saint-Laurent Warehouse
9,8,8,4.0,Cacao 70


In [20]:
testing_df = pd.read_csv("drive/My Drive/CMPE-256-Project/mon_rest_testing.csv")

In [21]:
testing_df.head(20)

,user_id_key,business_id_key,stars,business_name
0,14,18,4.0,Wendy's
1,15,19,4.0,Chez Ma Tante
2,21,9,4.0,Cacao 70
3,28,27,4.0,Pigeon
4,30,12,1.0,Les 3 Brasseurs
5,49,26,4.0,Les 3 Brasseurs
6,50,9,4.0,Cacao 70
7,53,12,3.0,Les 3 Brasseurs
8,63,26,2.0,Les 3 Brasseurs
9,65,9,3.0,Cacao 70


In [ ]:
##using Spark ML libraries for collaborative filtering

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [57]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True, userCol="user_id_key", itemCol="business_id_key", ratingCol="stars",
          coldStartStrategy="drop")
model = als.fit(training)

In [24]:
#creating testing dataframe with only userid and businessid as we need to predict ratings
testing_data=testing['user_id_key','business_id_key']

In [25]:
testing_data.head(5)

[Row(user_id_key=14, business_id_key=18),
 Row(user_id_key=15, business_id_key=19),
 Row(user_id_key=21, business_id_key=9),
 Row(user_id_key=28, business_id_key=27),
 Row(user_id_key=30, business_id_key=12)]

In [51]:
#applying trained model on testing data and evaluating predicted ratings
predictions = model.transform(testing_data)

In [44]:
testing_data.head(5)

[Row(user_id_key=14, business_id_key=18),
 Row(user_id_key=15, business_id_key=19),
 Row(user_id_key=21, business_id_key=9),
 Row(user_id_key=28, business_id_key=27),
 Row(user_id_key=30, business_id_key=12)]

In [52]:
predictions.take(5)

[Row(user_id_key=1372, business_id_key=26, prediction=-0.017919164150953293),
 Row(user_id_key=855, business_id_key=26, prediction=0.002780284732580185),
 Row(user_id_key=386, business_id_key=26, prediction=0.021576154977083206),
 Row(user_id_key=49, business_id_key=26, prediction=-0.08041803538799286),
 Row(user_id_key=314, business_id_key=26, prediction=0.027664655819535255)]

In [29]:
##create dataframe with original ratings for testing data
actual_test_ratings=testing['user_id_key','business_id_key','stars']

In [30]:
actual_test_ratings.head(5)

[Row(user_id_key=14, business_id_key=18, stars=4.0),
 Row(user_id_key=15, business_id_key=19, stars=4.0),
 Row(user_id_key=21, business_id_key=9, stars=4.0),
 Row(user_id_key=28, business_id_key=27, stars=4.0),
 Row(user_id_key=30, business_id_key=12, stars=1.0)]

In [53]:
##Set condition to join both dataframes the one with predicted ratings and one with actual ratings for test data
condi=[predictions.user_id_key == actual_test_ratings.user_id_key, predictions.business_id_key == actual_test_ratings.business_id_key]

In [54]:
##creating dataframe by performing inner join 
final_rating_testData = predictions.join(actual_test_ratings,condi,how='inner')

In [55]:
#visualizing final dataframe with predicted and actual ratings for test data
final_rating_testData.head(5)

[Row(user_id_key=1372, business_id_key=26, prediction=-0.017919164150953293, user_id_key=1372, business_id_key=26, stars=4.0),
 Row(user_id_key=855, business_id_key=26, prediction=0.002780284732580185, user_id_key=855, business_id_key=26, stars=5.0),
 Row(user_id_key=386, business_id_key=26, prediction=0.021576154977083206, user_id_key=386, business_id_key=26, stars=4.0),
 Row(user_id_key=49, business_id_key=26, prediction=-0.08041803538799286, user_id_key=49, business_id_key=26, stars=4.0),
 Row(user_id_key=314, business_id_key=26, prediction=0.027664655819535255, user_id_key=314, business_id_key=26, stars=4.0)]

In [56]:
##calculating rmse values using "rating" and "prediction" columns from final_rating_testData frame

evaluator = RegressionEvaluator( metricName="rmse", labelCol="stars",
                                predictionCol="prediction")
rmse = evaluator.evaluate(final_rating_testData)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 3.7545751674083623


The RMSE value for testing data is 3.75

In [39]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(2)

In [70]:
userRecs.head(50)

[Row(user_id_key=1580, recommendations=[Row(business_id_key=27, rating=2.544591188430786), Row(business_id_key=43, rating=2.511605739593506)]),
 Row(user_id_key=471, recommendations=[Row(business_id_key=7, rating=8.706377029418945), Row(business_id_key=56, rating=8.543800354003906)]),
 Row(user_id_key=1591, recommendations=[Row(business_id_key=26, rating=2.998678207397461), Row(business_id_key=50, rating=2.8784284591674805)]),
 Row(user_id_key=1342, recommendations=[Row(business_id_key=43, rating=6.3290629386901855), Row(business_id_key=68, rating=5.902369976043701)]),
 Row(user_id_key=2122, recommendations=[Row(business_id_key=14, rating=9.2449951171875), Row(business_id_key=42, rating=8.943242073059082)]),
 Row(user_id_key=2142, recommendations=[Row(business_id_key=14, rating=9.2449951171875), Row(business_id_key=42, rating=8.943242073059082)]),
 Row(user_id_key=463, recommendations=[Row(business_id_key=43, rating=8.387489318847656), Row(business_id_key=30, rating=3.997513771057129)]

In [87]:
df_new = spark_df[spark_df['user_id_key'] == 31] 

In [88]:
df_new.head(1)

[Row(user_id_key=31, business_id_key=9, stars=4.0, business_name='Cacao 70')]

In [90]:
df_test = training_df[training_df['business_id_key']==68]
df_test

,user_id_key,business_id_key,stars,business_name
1698,1408,68,2.0,Terra Verde
1799,1517,68,4.0,Terra Verde
1800,1518,68,5.0,Terra Verde
1801,1519,68,5.0,Terra Verde
1802,1520,68,5.0,Terra Verde
1803,1522,68,5.0,Terra Verde


In [ ]:
##saving predicted ratings in prediction.csv file for future reference
#using repartition() to save all the data samples in single csv file

csv_location = 'drive/My Drive/CMPE-256-Project/predictions_userALS.csv'
predictions.repartition(1).write.csv(path=csv_location, mode="append", header="true")